In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrindex.index_exo_base import IndexEXOBase
from datetime import datetime
from tmqrfeed.quotes import QuoteContFut
import pandas as pd
import numpy as np

In [ ]:
from bdateutil import relativedelta
from tmqr.logs import log
from tmqrfeed.quotes import QuoteContFut
from tmqrfeed import Costs
import pandas as pd
from tmqr.errors import SettingsError
import math


class EXOVolatilityAdjSpreadIndex(IndexEXOBase):
    _description_short = "EXO Vanilla Long/Short spread index"
    _description_long = ""
    _index_name = "EXOSpreadHedged"
    
    def __init__(self, datamanager, **kwargs):
        super().__init__(datamanager, **kwargs)
        
        self.PRIMARY_INSTRUMENT = self.context['primary_instrument']
        self.SECONDARY_INSTRUMENT = self.context['secondary_instrument']      
        
    @property
    def index_name(self):
        return f'{self.PRIMARY_INSTRUMENT}-{self.SECONDARY_INSTRUMENT}_{self._index_name}'
    
    def setup(self):
        
        #
        # IMPORTANT! Use trading session of self.PRIMARY_INSTRUMENT 
        #   All US.CL quotes and positions will use 'US.ES' decision and execution time
        # 
        self.dm.session_set(self.PRIMARY_INSTRUMENT, session_instance=self.session)
        
        #
        # Set primary quotes for 'US.ES' to align all data to its index
        #
        self.dm.series_primary_set(QuoteContFut, self.PRIMARY_INSTRUMENT,
                                   timeframe='D', decision_time_shift=self.decision_time_shift)
        
        self.dm.series_extra_set(self.SECONDARY_INSTRUMENT, QuoteContFut, self.SECONDARY_INSTRUMENT,
                                   timeframe='D', decision_time_shift=self.decision_time_shift)
        #
        # Set index costs (costs are calculated at the final stage, of index equirt calculation)
        # 
        self.dm.costs_set(self.PRIMARY_INSTRUMENT.split('.')[0], Costs(per_contract=self.costs_futures,
                                                               per_option=self.costs_options))
        
    
    def calc_exo_logic(self):
        """
        Calculates SmartEXO logic.
        NOTE: this method must use self.dm.quotes() or self.dm.quotes(series_key='for_secondary_series') to 
              calculate SmartEXO logic
        :return: Pandas.DataFrame with index like in dm.quotes() (i.e. primary quotes)
        """
        # Getting quotes
        primary_quotes = self.dm.quotes()
        secondary_quotes = self.dm.quotes(self.SECONDARY_INSTRUMENT)
        
        # Getting instrument information 
        primary_instrument_info = self.dm.instrument_info_get(self.PRIMARY_INSTRUMENT)
        secondary_instrument_info = self.dm.instrument_info_get(self.SECONDARY_INSTRUMENT)
        
        # Calculating point value
        primary_instrument_point_value = 1.0 / primary_instrument_info.ticksize * primary_instrument_info.tickvalue
        secondary_instrument_point_value = 1.0 / secondary_instrument_info.ticksize * secondary_instrument_info.tickvalue
        
        # Calculating hedge ratios
        hedge_type = self.context.get('hedge_type', '')
        hedge_window = self.context.get('hedge_window', None)
        
        if hedge_type == 'vola':
            # Volatility based hedging
            primary_usd_vola = (primary_quotes['h']-primary_quotes['l']).rolling(hedge_window).median() * primary_instrument_point_value
            secondary_usd_vola = (secondary_quotes['h']-secondary_quotes['l']).rolling(hedge_window).median() * secondary_instrument_point_value
            
            # Calculating USD value ratio per 10 contracts        
            hedge_ratio = primary_usd_vola / secondary_usd_vola
        elif hedge_type == 'beta':
            # Beta based hedging
            rets_primary = primary_quotes['c'].diff() * primary_instrument_point_value
            rets_secondary = secondary_quotes['c'].diff() * secondary_instrument_point_value

            sigma_primary = rets_primary.rolling(hedge_window).std()
            sigma_secondary = rets_secondary.rolling(hedge_window).std()

            cor = rets_primary.rolling(hedge_window).corr(rets_secondary)
            
            # Calculating hedge ratio, i.e. BETA = Correlation(A, B) * (StDev(A) / StDev(B))
            hedge_ratio = cor * (sigma_primary / sigma_secondary)
        else:
            raise SettingsError(f"Unexpected hedge type: '{hedge_type}', only 'vola' and 'beta' allowed")
        
        
        
        
        
        # Add extra logic to illustrate how SmartEXO can be implemeted
        # Define bull trend regime as close of primary > moving_average(primary, 20-periods)
        primary_in_bull_trend = primary_quotes['c'] > primary_quotes['c'].rolling(20).mean()
        
        # We have to return pandas.DataFrame class
        return pd.DataFrame({
            'hedge_ratio': hedge_ratio,
            # Include SMART EXO regime
            'primary_in_bull_trend': primary_in_bull_trend,
        })
    
    def manage_position(self, dt, pos, logic_df):
        """
        Manages opened position (rollover checks/closing, delta hedging, etc)
        :param dt: current datetime
        :param pos: Position instance
        :param logic_df: result of calc_exo_logic()[dt]  if applicable
        :return: nothing, manages 'pos' in place
        """        
        #
        # Check expiration moment
        # Or you can check custom days to expiration values
        #  pos.almost_expired_ratio(dt, rollover_days_before_fut=5, rollover_days_before_opt=7)
        if pos.almost_expired_ratio(dt) > 0:                        
            pos.close(dt)
            
        #
        # !!! Uncomment position management logic if you need to implement SmartEXO style index
        #
            
        #
        # Check business days after last transaction
        #
        #pos_last_transaction_date = pos.last_transaction_date(dt)        
        #log.debug("Last transaction date: {0}".format(pos_last_transaction_date))
        #days_after_last_trans = relativedelta(dt, pos_last_transaction_date).bdays
        # 
        #if days_after_last_trans > 7:
        #    log.debug("Business days > 7, closing position")
        #    pos.close(dt)
        #    return 
        
        #
        # Delta based rebalance
        #
        #delta = pos.delta(dt)
        #if delta > 0.7:
        #    log.debug("Delta > 0.7")
        #    pos.close(dt)
        #    return 
                
        #
        # logic_df based rebalance
        #
        #primary_in_bull_trend = logic_df['primary_in_bull_trend']
        #
        #if not primary_in_bull_trend: 
        #    log.debug("not primary_in_bull_trend")
        #    pos.close(dt)
        #    return 

    def construct_position(self, dt, pos, logic_df):
        """
        EXO position construction method
        
        NOTE!: this method only called when there is no active position for 'dt'
        :param dt: current datetime
        :param pos: Position instance
        :param logic_df: result of calc_exo_logic()[dt]  if applicable
        :return: nothing, manages 'pos' in place
        """
        # Getting active futures and options chains
        #fut_primary, opt_chain_primary = self.dm.chains_options_get(self.PRIMARY_INSTRUMENT, dt)
        #fut_secondary, opt_chain_secondary = self.dm.chains_options_get(self.SECONDARY_INSTRUMENT, dt)
        
        fut_primary = self.dm.chains_futures_get(self.PRIMARY_INSTRUMENT, dt, offset=0)
        fut_secondary = self.dm.chains_futures_get(self.SECONDARY_INSTRUMENT, dt, offset=0)
        
        # Aligning spread legs expirations
        if fut_primary.expiration_month != fut_secondary.expiration_month:
            if fut_primary.expiration < fut_secondary.expiration:
                fut_primary = self.dm.chains_futures_get(self.PRIMARY_INSTRUMENT, dt, offset=1)
            else:
                fut_secondary = self.dm.chains_futures_get(self.SECONDARY_INSTRUMENT, dt, offset=1)
                
            assert fut_primary.expiration_month == fut_secondary.expiration_month
            
        
        
        hedge_ratio = logic_df['hedge_ratio']
        primary_in_bull_trend = logic_df['primary_in_bull_trend']
        
        if math.isnan(hedge_ratio) or hedge_ratio == 0:
            # Skipping 'NaN' hedge ratios (usualy days before full hedge_window)
            log.debug(f"{dt}: Skipping 'NaN' or zero hedge ratios")
            return
        
        # PRIMARY long
        primary_qty = 10.0 
        pos.add_transaction(dt, fut_primary, primary_qty)
        
        # SECONDARY short
        secondary_qty = round(primary_qty * hedge_ratio)
        pos.add_transaction(dt, fut_secondary, -secondary_qty)
        
        
        # Usign SmartEXO hedge style
        #if primary_in_bull_trend:
        #    # Add primary long call if primary_in_bull_trend
        #    target_delta = 0.4
        #    hedge_qty = round(primary_qty * target_delta)
        #    
        #    pos.add_transaction(dt, opt_chain_primary.find(dt, target_delta, 'C', how='delta'), hedge_qty)
        

In [ ]:
INDEX_CONTEXT = {
    'context': {
        'primary_instrument': 'US.ES',
        'secondary_instrument': 'US.ZN',
        
        'hedge_type': 'vola', # 'vola' - median(h-l) volatility hedge, 'beta' - Beta coef. hedge,
        'hedge_window': 220,  # number of periods to calculate hedge coef. 
        'costs_futures': 3.0,
        'costs_options': 3.0,
    }
}
dm = DataManager()
index = EXOVolatilityAdjSpreadIndex(dm, **INDEX_CONTEXT)

#
# BOTH index init code lines are equal
#

#index = EXODeltaTargetGeneric(dm, instrument="US.ES", costs_futures=3.0, costs_options=3.0)

In [ ]:
index.run()

## Index equity

In [ ]:
list(index.data.columns)

# Spread position

Both of ES and CL have the same decision time and price



In [ ]:
index.position

In [ ]:
index.data['equity_execution'].plot()
title(index.index_name)

In [ ]:
index.index_name

In [ ]:
index.data['equity_execution'].tail()